In [ ]:
import os
import chromadb
from google import genai
from huggingface_hub import InferenceClient

CHROMA_HOST = os.getenv("CHROMA_HOST")
CHROMA_PORT = int(os.getenv("CHROMA_PORT"))

HF_MODEL = "sentence-transformers/all-mpnet-base-v2"
HF_TOKEN = os.environ["HF_TOKEN"]

chroma_client = chromadb.HttpClient(host=CHROMA_HOST, port=CHROMA_PORT)
collection = chroma_client.get_or_create_collection("superheroes")

gemini_client = genai.Client()

hf_client = InferenceClient(provider="auto", api_key=HF_TOKEN)

In [36]:
def embed_text(query: str):
    return hf_client.feature_extraction([query], model=HF_MODEL)[0]

In [ ]:
def ask_superheroes(question: str, top_k: int = 50) -> str:
    query_vector = embed_text(question)

    results = collection.query(
        query_embeddings=[query_vector],
        n_results=top_k
    )
    docs = results["documents"][0]
    context = "\n\n".join(docs)

    prompt = (
        "You are batman's AI. Use the context below to answer the question. "
        "If the answer is not in the context, improvise, otherwise, say 'I don't know'.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\nAnswer:"
    )

    response = gemini_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    return response.text

In [44]:
# 1. Make sure these exist in your script:
# - `collection` → Chroma collection with superhero docs + embeddings
# - `embed_text(text: str)` → returns HF embedding vector
# - `gemini_client` → from google.genai.Client()

# 2. Ask a question
question = "which robin has the highest stats, also count there adult versions(such as nightwing for dick grayson)?"
answer = ask_superheroes(question)

print("Q:", question)
print("A:", answer)

Q: which robin has the highest stats, also count there adult versions(such as nightwing for dick grayson)?
A: Considering their adult versions, **Dick Grayson** (the first Robin) has the highest overall stats.

As **Nightwing**, his stats are:
*   Intelligence: 88.0
*   Strength: 11.0
*   Speed: 33.0
*   Durability: 28.0
*   Power: 36.0
*   Combat: 100.0

These are the same as his stats as **Batman II**.
